1. data 16khz 로 바꾸기
2.  normalize the input to 0 mean and 0.5 std
* You can check ast/src/get_norm_stats.py to see how we compute the stats, or you can try using our AudioSet normalization input_spec = (input_spec + 4.26) / (4.57 * 2)

* Please note that AST needs smaller learning rate

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers[torch] datasets[audio] audiomentations

In [3]:
!pip install torch torchaudio

In [4]:
!pip install "transformers[sentencepiece]"

In [5]:
!pip install --upgrade torch torchvision torchaudio transformers


In [6]:
# from transformers import AutoFeatureExtractor, ASTModel
# import torch
# import torchaudio

# model_name = "MIT/ast-finetuned-audioset-10-10-0.4593"
# sr = 16000
# extractor = AutoFeatureExtractor.from_pretrained(model_name,
#                                                  sampling_rate = sr, #입력 16khz로 맞추기
#                                                 do_normalize = True) #normalize진행
# model = ASTModel.from_pretrained(model_name)
# model.eval()

# def preprocess_ACT(audio_files, sampling_rate = 16000, min_length =400):
#   embeddings = []

#   for audio_file in audio_files:
#     audio, _ = torchaudio.load(audio_file)


#     input_values = extractor(audio)["input_values"]
#     with torch.no_grad():
#       output = model(input_values).last_hidden_state
#     embeddings.append(output.squeeze().cpu().numpy())

#   return embeddings


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

# 확인

* transform 없애버림
* dataloader할때 애초에 sr 16khz로 설정

In [20]:
'''Contrast learning을 위해서 오디오 pair를 만듦
-> 오디오의 전반과 후반 각각에서 원하는 길이의 연속된 오디오 구간 추출'''

# ------------
# AudioDataset
# ------------
import os
import torchaudio
from torch.utils.data import Dataset

class AudioDataset(Dataset):
    def __init__(self, file_paths):
        """
        Contrastive Learning용 오디오 데이터셋.
        Args:
            file_paths (list): 오디오 파일 경로 리스트.
            transform (callable, optional): 데이터 증강을 위한 함수.
        """
        self.file_paths = file_paths


    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        # 오디오 파일 로드
        file_path = self.file_paths[idx]
        waveform, sample_rate = torchaudio.load(file_path)

        #일단 여기서부터 resampling , sr = 16khz
        if sample_rate != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
            waveform = resampler(waveform)

        #음악 id만 추출 (파일 이름에서)
        track_id = os.path.basename(file_path).replace('.wav', '')

        return waveform, track_id

# ------------
# create_contrastive_datasets
# ------------
from sklearn.model_selection import train_test_split

def create_contrastive_datasets(dataset_dir, train_ratio=0.8, random_seed=42):
    """
    오디오 파일을 train/val로 나누고, 데이터셋 생성.
    Args:
        dataset_dir (str): 오디오 파일이 포함된 디렉토리.
        train_ratio (float): train 데이터의 비율.
        random_seed (int): 랜덤 시드 값.
        transform (callable, optional): 데이터 증강 함수.
    Returns:
        train_dataset (AudioDataset): 학습 데이터셋.
        val_dataset (AudioDataset): 검증 데이터셋.
    """
    # 파일 경로 읽기
    file_paths = [
        os.path.join(dataset_dir, f)
        for f in os.listdir(dataset_dir)
    ]

    # Train/Validation Split
    train_files, val_files = train_test_split(
        file_paths, train_size=train_ratio, random_state=random_seed
    )

    # Dataset 생성
    train_dataset = AudioDataset(train_files)
    val_dataset = AudioDataset(val_files)

    return train_dataset, val_dataset


# ------------
# ContrastiveDataset
# ------------
import torch
from torch import Tensor
from torch.utils.data import Dataset
from typing import Tuple, List
import random

class ContrastiveDataset(Dataset):
    def __init__(self, dataset: Dataset, input_shape: List[int]):
        self.dataset = dataset
        self.input_shape = input_shape
        self.ignore_idx = []

    def __len__(self) -> int:
        return len(self.dataset)

    def _get_continuous_clip(self, audio: Tensor) -> Tensor:
        """오디오에서 연속된 길이의 클립을 추출하는 메서드"""
        clip_length = self.input_shape[1]  # 원하는 클립 길이
        max_start_idx = audio.shape[1] - clip_length
        start_idx = random.randint(0, max_start_idx)  # 연속된 구간을 선택하기 위해 랜덤 인덱스 선택
        clip = audio[:, start_idx:start_idx + clip_length]


        return clip

    def __getitem__(self, idx) -> Tuple[Tensor, Tensor]:
        if idx in self.ignore_idx:
            return self[idx + 1]

        audio, track_id = self.dataset[idx]

        if audio.shape[1] < self.input_shape[1]:  #오디오 길이가 짧으면 제외
            self.ignore_idx.append(idx)
            return self[idx + 1]



        # 전반부와 후반부로 나누고, 각각 연속된 클립을 선택하여 pair 생성
        mid_point = audio.shape[1] // 2
        first_half = audio[:, :mid_point]
        second_half = audio[:, mid_point:]

        # 전반부와 후반부에서 각각 연속된 클립 추출
        clip_a = self._get_continuous_clip(first_half)
        clip_b = self._get_continuous_clip(second_half)

        return clip_a, clip_b, track_id




In [22]:

# ------------
# dataloaders
# ------------


import torch
from torch.utils.data import DataLoader



# 데이터 경로 설정
dataset_dir = "/content/drive/MyDrive/mp3_dl_2024S_course/move"

# 오디오 파일 경로 및 데이터셋 준비
train_dataset, val_dataset = create_contrastive_datasets(dataset_dir, train_ratio=0.8)

# ContrastiveDataset으로 변환
sample_rate = 16000 # [1, sample_rate*30]: 30초로 구간 설정
train_contrastive_dataset = ContrastiveDataset(train_dataset, input_shape=[1, sample_rate*30])
val_contrastive_dataset = ContrastiveDataset(val_dataset, input_shape=[1, sample_rate*30])

# DataLoader로 배치 생성
train_loader = DataLoader(train_contrastive_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_contrastive_dataset, batch_size=16, shuffle=False)
# -> 한 배치의 구성 : clip_a, clip_b, file_id


In [25]:
# 데이터셋에서 몇 개의 예시 가져오기
for i in range(5):  # 첫 5개 데이터 예시 출력
    waveform, track_id = train_dataset[i]
    print(f"Track Id: {track_id}")
    print(f"Waveform shape: {waveform.shape}")  # [채널 수, 샘플 수]

Track Id: TRPKRJU128E07935F6
Waveform shape: torch.Size([2, 2524636])
Track Id: TRVHERE128F930FD80
Waveform shape: torch.Size([2, 4066279])
Track Id: TRQWMTI128F4262442
Waveform shape: torch.Size([2, 1865190])
Track Id: TRLLRSB128F92F543F
Waveform shape: torch.Size([2, 2053550])
Track Id: TRDMZYI12903CF3A27
Waveform shape: torch.Size([2, 4071643])


In [42]:
from transformers import AutoFeatureExtractor, ASTModel
import torch
import torchaudio

class ASTEncoder:
    def __init__(self, model_name="MIT/ast-finetuned-audioset-10-10-0.4593", sampling_rate=16000):
        self.sampling_rate = sampling_rate
        self.extractor = AutoFeatureExtractor.from_pretrained(model_name, sampling_rate=sampling_rate, do_normalize=True)  # 입력 16khz로 맞추기, normalize 진행
        self.model = ASTModel.from_pretrained(model_name)
        self.model.eval()

    def preprocess(self, clip_a, clip_b):
        clip_a_embeddings = []
        clip_b_embeddings = []

        for audio in clip_a:
          #두 채널인 경우 두 채널 모두 처리 : 스트레오 오디오면 2개의 좌우채널 존재
          if audio.ndim >1:
            channel_embeddings = []
            for chanel in audio:
              #텐서를 numpy로 변환
              audio_np = chanel.numpy()

              #특징 추출
              input_values = self.extractor(audio_np, sampling_rate=self.sampling_rate)["input_values"]

              if isinstance(input_values, list):
                input_values = input_values[0]
              #numpy 배열로 확실히 변환
              input_values = np.array(input_values)

              #텐서변환
              input_values = torch.from_numpy(input_values)

              # 입력 텐서의 모양 확인
              print("Before Unsqueeze Input Values Shape:", input_values.shape)

              # unsqueeze를 통해 차원 추가
              input_values = input_values.unsqueeze(0)  # (1, max_length, num_mel_bins) 형태로 변환

              # 입력 텐서의 모양 확인
              print("After Unsqueeze Input Values Shape:", input_values.shape)

              with torch.no_grad():
                output = self.model(input_values).last_hidden_state
              channel_embeddings.append(output.squeeze().cpu().numpy())

            clip_a_embeddings.append(np.mean(channel_embeddings, axis=0))

        for audio in clip_b:
          #두 채널인 경우 두 채널 모두 처리 : 스트레오 오디오면 2개의 좌우채널 존재
          if audio.ndim >1:
            channel_embeddings = []
            for chanel in audio:
              #텐서를 numpy로 변환
              audio_np = chanel.numpy()

              #특징 추출
              input_values = self.extractor(audio_np, sampling_rate=self.sampling_rate)["input_values"]

              if isinstance(input_values, list):
                input_values = input_values[0]

              #numpy 배열로 확실히 변환
              input_values = np.array(input_values)

              #텐서변환
              input_values = torch.from_numpy(input_values)

              # 입력 텐서의 모양 확인
              print("Before Unsqueeze Input Values Shape:", input_values.shape)

              # unsqueeze를 통해 차원 추가
              input_values = input_values.unsqueeze(0)  # (1, max_length, num_mel_bins) 형태로 변환

              # 입력 텐서의 모양 확인
              print("After Unsqueeze Input Values Shape:", input_values.shape)

              with torch.no_grad():
                output = self.model(input_values).last_hidden_state
              channel_embeddings.append(output.squeeze().cpu().numpy())

            clip_b_embeddings.append(np.mean(channel_embeddings, axis=0))

        return clip_a_embeddings, clip_b_embeddings

input_values (torch.FloatTensor of shape (batch_size, max_length, num_mel_bins)) — Float values mel features extracted from the raw audio waveform. Raw audio waveform can be obtained by loading a .flac or .wav audio file into an array of type List[float] or a numpy.ndarray, e.g. via the soundfile library (pip install soundfile). To prepare the array into input_features, the AutoFeatureExtractor should be used for extracting the mel features, padding and conversion into a tensor of type torch.FloatTensor. See call()

In [43]:
import numpy as np

ast_encoder = ASTEncoder()

batch_size = 10
for batch in train_loader:
  clip_a, clip_b, track_ids = batch  # 오디오와 file_ids 로드

  print("Clip A Shape:", clip_a.shape)
  print("Clip B Shape:", clip_b.shape)
  print("Music IDs:", track_ids)



  # 1) 오디오 임베딩 생성 (오디오는 로스 계산에만 사용)
  clip_a_embeddings, clip_b_embeddings = ast_encoder.preprocess(clip_a, clip_b)

  # 2) 임베딩 확인
  print("Audio Embeddings A:")
  print(np.array(clip_a_embeddings))
  print(np.array(clip_a_embeddings).shape)
  print("Audio Embeddings B:")
  print(np.array(clip_b_embeddings))
  print(np.array(clip_b_embeddings).shape)


  # track_ids와 함께 저장 예시
  for i, track_id in enumerate(track_ids):
      print(f"Track ID: {track_id}")
      print(f"Clip A Embedding: {clip_a_embeddings[i]}")
      print(f"Clip B Embedding: {clip_b_embeddings[i]}")

  break

Clip A Shape: torch.Size([16, 2, 480000])
Clip B Shape: torch.Size([16, 2, 480000])
Music IDs: ('TRXNKNN128F9319C21', 'TRQQEEF128F92D5BC4', 'TRYFHTO128F9319BA8', 'TRRMTOK12903CE577E', 'TRFKWVQ128F9341AB2', 'TRIFICJ128F426A83C', 'TRIXEFK128F14710B2', 'TRDUUZR128F4263FCB', 'TRMIDHA128F931B196', 'TRWEJNA128F930D40D', 'TRGJWNK128F9351E94', 'TRVHERE128F930FD80', 'TREYRWO128F424802A', 'TRHOHFZ12903CA8562', 'TRSBFZV128F9312CBB', 'TRNDFGY128F148D04D')
Before Unsqueeze Input Values Shape: torch.Size([1024, 128])
After Unsqueeze Input Values Shape: torch.Size([1, 1024, 128])
Before Unsqueeze Input Values Shape: torch.Size([1024, 128])
After Unsqueeze Input Values Shape: torch.Size([1, 1024, 128])
Before Unsqueeze Input Values Shape: torch.Size([1024, 128])
After Unsqueeze Input Values Shape: torch.Size([1, 1024, 128])
Before Unsqueeze Input Values Shape: torch.Size([1024, 128])
After Unsqueeze Input Values Shape: torch.Size([1, 1024, 128])
Before Unsqueeze Input Values Shape: torch.Size([1024, 1